In [42]:
# Урок 3. Парсинг данных. HTML, Beautiful Soap
# Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. 
# Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
# vv Наименование вакансии.
# vv Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
# vv Ссылку на саму вакансию.
# Сайт, откуда собрана вакансия.
# По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
# Структура должна быть одинаковая для вакансий с всех сайтов. 
# Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [43]:
# Функция парсит вакансии с hh.ru

def fParseHhRu(vacancies_list, vacancy_text, pagenum):
    url = 'https://hh.ru/search/vacancy'
    params = {  'area': 1,
                'text': vacancy_text,
                'items_on_page': 20,
                'experience': 'noExperience',
                'page': pagenum
             }
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    
    res = requests.get(url=url, params=params, headers=headers) 
    soup = bs(res.text, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'serp-item'})
    #vacancies_list = []

    for vacancy in vacancies:
        vacancy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
        vacancy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
        vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        vacancy_employer = vacancy.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'}).getText()
        vacancy_adress = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
        if vacancy_salary is not None:
                vacancy_salary = vacancy_salary.getText().replace('\u202f','')
        vacancy_dict = {  'source': 'hh.ru',
                          'name': vacancy_name,
                          'link': vacancy_link,
                          'salary': vacancy_salary,
                          'employer': vacancy_employer,
                          'adress': vacancy_adress
                       }
        vacancies_list.append(vacancy_dict)
    #return vacancies_list

In [44]:
# Функция парсит вакансии с rabota.ru

def fParseRabotaRu(vacancies_list, vacancy_text, pagenum):
    url = 'https://www.rabota.ru/vacancy/'
    params = { 'query': vacancy_text,
               'sort': 'relevance',
               'page': pagenum
             }
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

    res = requests.get(url=url, params=params, headers=headers) 
    soup = bs(res.text, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'vacancy-preview-card__top'})
    
    for vacancy in vacancies:
            vacancy_name = vacancy.find('a', {'class': 'vacancy-preview-card__title_border'}).getText()
            vacancy_link = vacancy.find('a', {'class': 'vacancy-preview-card__title_border'})['href']
            if "https://" not in vacancy_link:
                vacancy_link = "https://rabota.ru" + vacancy_link
            vacancy_salary = vacancy.find('div', {'class': 'vacancy-preview-card__salary'}).find('a')
            vacancy_employer = vacancy.find('a', {'itemprop': 'name'}).getText()
            vacancy_adress = vacancy.find('span', {'class': 'vacancy-preview-location__address-text'}).getText()
   
            if vacancy_salary is not None:
                vacancy_salary = vacancy_salary.getText().replace('&nbsp;','')

            vacancy_dict = { 'source': 'rabota.ru',
                             'name': vacancy_name.replace('  ','').replace('\n',''),
                             'link': vacancy_link,
                             'salary': vacancy_salary,
                             'employer': vacancy_employer.replace('  ','').replace('\n',''),
                             'adress': vacancy_adress.replace('  ','').replace('\n','')
                           }
            vacancies_list.append(vacancy_dict)

In [45]:
# запросы к сайтам и формирование списка вакансий

vacancies_list = []
vacancy_text='python'

fParseHhRu(vacancies_list,vacancy_text,0)
fParseHhRu(vacancies_list,vacancy_text,1)
fParseHhRu(vacancies_list,vacancy_text,2)

fParseRabotaRu(vacancies_list,vacancy_text,1)
fParseRabotaRu(vacancies_list,vacancy_text,2)
fParseRabotaRu(vacancies_list,vacancy_text,3)

for v in vacancies_list:
    print(v)
    

{'source': 'hh.ru', 'name': 'Аналитик (Консультант-разработчик на Optimacros) (Junior)', 'link': 'https://hh.ru/vacancy/69608845?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': 'от 50000 руб.', 'employer': 'Human Line', 'adress': 'Москва, Библиотека им.Ленина'}
{'source': 'hh.ru', 'name': 'Стажер-аналитик (большие данные)', 'link': 'https://hh.ru/vacancy/70704887?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': None, 'employer': 'Яндекс', 'adress': 'Москва'}
{'source': 'hh.ru', 'name': 'Стажер разработчик BI', 'link': 'https://hh.ru/vacancy/70355093?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': None, 'employer': 'билайн: Стажировки', 'adress': 'Москва'}
{'source': 'hh.ru', 'name': 'Quantitative Researcher', 'link': 'https://hh.ru/vacancy/70118714?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=python', 'salary': 'от 300000 руб.', 'employer': 'Gara.capital', 'adress': 'Москва'}
{'so

In [47]:
# вывод списка в csv файл
vacancies_df = pd.DataFrame(vacancies_list)
vacancies_df.to_csv('vacancies_list.csv')